<a href="https://colab.research.google.com/github/ansharyis/ml-colab-project/blob/main/notebooks/01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
DATA_DIR = "/content/drive/MyDrive/Machine Learning Group Work"


In [2]:
# Project: ML Weight Prediction
# Notebook: ....
# Owner: ....
# Description: ....

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

train_df = pd.read_excel(f"{DATA_DIR}/RAW/training_dataset.xlsx")
vars_df  = pd.read_excel(f"{DATA_DIR}/RAW/nhis variable names 2020.xlsx")

print(train_df.shape)
print(vars_df.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Machine Learning Group Work/RAW/training_dataset.xlsx'